In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Definisikan library dan Modul

In [ ]:
# Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import itertools
import tensorflow as tf
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Modul
warnings.filterwarnings("ignore", category=ConvergenceWarning,
                        module="sklearn")

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.YlGn):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Definisikan file

In [ ]:
file = pd.read_csv('../input/heart-disease-cleveland-uci/heart_cleveland_upload.csv')
col = file.columns
df_data = pd.DataFrame(file, index=None)
np_data = np.array(file)
df_data.head(10)

# Analisa Jumlah Label target

In [ ]:
kondisi = ['Sehat', 'Sakit']
jumlah = [np_data[:,-1][np_data[:,-1]==0].shape[0], np_data[:,-1][np_data[:,-1]==1].shape[0]]
plt.bar(kondisi,jumlah, color=['seagreen', 'red'])
plt.show()
print(f"Sehat: {jumlah[0]}, Sakit: {jumlah[1]}")

# Normalisasi dan Partisi Fitur

In [ ]:
fitur = []
target = []
fitur = StandardScaler().fit_transform(np_data)
# fitur = fitur[:,:-1].reshape(297, 13, 1)
target = tf.keras.utils.to_categorical(fitur[:,-1])
x_train, x_test, y_train, y_test = train_test_split(fitur[:,:-1], target, test_size=0.2, shuffle = True, random_state=1)

# Model

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=[72,39],solver='sgd',learning_rate_init= 0.01, max_iter=100)

# Train the model
mlp.fit(x_train, y_train)
print(f'Accuracy Training: {round(mlp.score(x_train, y_train),4)*100}%')

# Test the model
y_pred = mlp.predict(x_test)

# Confusion Matrix
cx=confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
plot_confusion_matrix(cx,classes=['Sehat', 'Sakit'], normalize=False)
plt.show()
# Accuracy
accuracy=accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
# Precision
precision=precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='binary')
# Recall
recall=recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='binary')

print(f"accuracy: {round(accuracy,4)*100}%")
print(f"precision: {round(precision,4)*100}%")
print(f"recall: {round(recall,4)*100}%")

# Model Keras

In [ ]:
import numpy as np
import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Build the model.
model = Sequential([
  Dense(72, activation='relu', input_shape=(237,13)),
  Dense(39, activation='relu'),
  Dense(2, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
)

# Train the model.
model.fit(
  x_train,
  y_train,
  epochs=50,
  batch_size=39,
)

# Test the model
y_pred = model.predict(x_test)

# Confusion Matrix
cx=confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
plot_confusion_matrix(cx,classes=['Sehat', 'Sakit'], normalize=False)
plt.show()
# Accuracy
accuracy=accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
# Precision
precision=precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='binary')
# Recall
recall=recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='binary')

print(f"accuracy: {round(accuracy,4)*100}%")
print(f"precision: {round(precision,4)*100}%")
print(f"recall: {round(recall,4)*100}%")